In [20]:
%%capture

!pip install gdown gpxpy

In [5]:
%%capture

import gdown

FILE_URL = 'https://drive.google.com/uc?id=1nqGQiFVmub2mDNCx0kcLp1uoRXi19qLg'
gdown.download(FILE_URL, output='gnss-logger-fix.csv', quiet=True)

In [13]:
import pandas as pd

gnss_logger_df = pd.read_csv('/content/gnss-logger-fix.csv')
gnss_logger_df['Time'] = pd.to_datetime(gnss_logger_df['UnixTimeMillis'], unit='ms', utc=True)
gnss_logger_df.head()

,Fix,Provider,LatitudeDegrees,LongitudeDegrees,AltitudeMeters,SpeedMps,AccuracyMeters,BearingDegrees,UnixTimeMillis,SpeedAccuracyMps,BearingAccuracyDegrees,elapsedRealtimeNanos,VerticalAccuracyMeters,MockLocation,NumberOfUsedSignals,VerticalSpeedAccuracyMps,SolutionType,Time
0,Fix,FLP,20.993869,105.868808,-18.700001,0.811977,13.782000,261.85977,1732631567781,0.197464,45.0,7192350787648816,6.809736,0,NaN,NaN,NaN,2024-11-26 14:32:47.781000+00:00
1,Fix,GPS,20.993706,105.868735,-10.960552,0.000000,11.770008,NaN,1732631567000,20.000000,NaN,7192351749849000,6.352025,0,NaN,NaN,NaN,2024-11-26 14:32:47+00:00
2,Fix,FLP,20.993860,105.868798,-18.700001,0.797945,13.385000,257.30023,1732631568782,0.174914,45.0,7192351787648816,6.820159,0,NaN,NaN,NaN,2024-11-26 14:32:48.782000+00:00
3,Fix,FLP,20.993850,105.868791,-18.700001,0.789725,12.889000,255.06831,1732631569266,0.221858,45.0,7192352272836881,6.820159,0,NaN,NaN,NaN,2024-11-26 14:32:49.266000+00:00
4,Fix,NLP,20.993993,105.868684,-18.600000,NaN,23.797000,NaN,1732631569211,NaN,NaN,7192352217000000,3.046379,0,NaN,NaN,NaN,2024-11-26 14:32:49.211000+00:00


In [19]:
%%capture

import gdown

FILE_URL = 'https://drive.google.com/uc?id=1exVZ8rXLS1EpmZcbzsdIPyc2IkKb7yet'
gdown.download(FILE_URL, output='osmtracker.gpx', quiet=True)

In [23]:
import gpxpy
import pandas as pd

def parse_gpx(filepath):
  gpx_file = open(filepath, 'r')
  gpx = gpxpy.parse(gpx_file)

  data = []

  for track in gpx.tracks:
    for segment in track.segments:
      for point in segment.points:
        time, lat, lng = point.time, point.latitude, point.longitude,
        data.append({
          'time': time,
          'lat': lat,
          'lon': lng,
        })

  df = pd.DataFrame(data)
  df['time'] = pd.to_datetime(df['time'])

  return df

osmtracker_df = parse_gpx('/content/osmtracker.gpx')
osmtracker_df.head()

,time,lat,lon
0,2024-11-26 14:32:47+00:00,20.993706,105.868735
1,2024-11-26 14:32:48+00:00,20.993707,105.868711
2,2024-11-26 14:32:49+00:00,20.993702,105.868678
3,2024-11-26 14:32:50+00:00,20.993695,105.868643
4,2024-11-26 14:32:51+00:00,20.993696,105.868611


In [30]:
import folium

def plot_points(center, osmtracker_points, gnss_logger_points):
  m = folium.Map(
      location=center,
      zoom_start=18,
  )

  folium.PolyLine(osmtracker_points, color="black", weight=2.5, opacity=0.7).add_to(m)
  folium.PolyLine(gnss_logger_points, color="orange", weight=5.5, opacity=0.5).add_to(m)

  return m

osmtracker_points = osmtracker_df[['lat', 'lon']].values.tolist()
gnss_logger_points = gnss_logger_df[gnss_logger_df['Provider'] == 'GPS'][['LatitudeDegrees', 'LongitudeDegrees']].values.tolist()

plot_points(
    (osmtracker_df['lat'].mean(), osmtracker_df['lon'].mean()),
    osmtracker_points,
    gnss_logger_points,
)